In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import math
import re

In [2]:
def is_number(s):
    """ Returns True is string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

### Data Cleaning

In [4]:
def is_job_status_and_living_area_reversed(x):
    return ((x["job_status"] != None and x["job_status"] in ("r", "c", "city", "remote", "remotee"))
             or (x["living_area"] != None and x["living_area"] in ("private_sector", "business_owner")))
def process_job_status(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x in ("private sector", "privattte", "private", "private_sector"):
        return "private_sector"
    elif x in ("government", "govt."):
        return "government"
    elif x in ("business_owner", "business owner", "biz"):
        return "business_owner"
    elif x in ("parental_leave", "parental leave"):
        return "parental_leave"
    else:
        return x
    
def process_living_area(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x == 'c':
        return 'city'
    elif x in ('r', 'remotee'):
        return 'remote'
    else:
        return x

def split_job_status_and_living_area(x):
    pair = x.lower().split("?") if x != None else [x, x]
    if len(pair) < 2:
        pair = [pair[0], None]
    return pair

def process_job_status_and_living_area(df):
    df["job_status"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[0])
    df["living_area"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[1])
    job_status = df.apply(lambda x: x["living_area"] if is_job_status_and_living_area_reversed(x) else x["job_status"], 1)
    living_area = df.apply(lambda x: x["job_status"] if is_job_status_and_living_area_reversed(x) else x["living_area"], 1)
    df["job_status"] = job_status.apply(lambda x: process_job_status(x))
    df["living_area"] = living_area.apply(lambda x: process_living_area(x))

In [5]:
process_job_status_and_living_area(train)
process_job_status_and_living_area(test)

In [6]:
def process_smoker_status(x):
    if x == None:
        return None
    elif x.startswith("non"):
        return "non-smoker"
    elif x.startswith("quit"):
        return "quit"
    elif x.startswith("active"):
        return "active_smoker"
    else:
        return None
train["smoker_status"] = train["smoker_status"].astype(str).apply(process_smoker_status)
test["smoker_status"] = test["smoker_status"].astype(str).apply(process_smoker_status)

Convert BMI to numeric

In [7]:
train["BMI"] = pd.to_numeric(train["BMI"],errors="coerce")

Treat '.,' as Missing Values

In [8]:
train[train["high_BP"] == '.,'] = None

Clean Sex and Age

In [ ]:
def clean_sex_age(sex_age_list):
    if type(sex_age_list) is not list:
        return [None, None]
    # Strip and Upper case both sex and age
    sex_age_list[0],sex_age_list[1] = sex_age_list[0].strip().upper(), sex_age_list[1].strip().upper()
    
    # 2nd : first one is empty and second one is not numeric
    if (is_number(sex_age_list[0]) or (not sex_age_list[0] and not is_number(sex_age_list[1]))): 
        sex_age_list = sex_age_list[::-1]
    sex = sex_age_list[0].strip().upper()
    
    # Take
    if sex in ('FEMALE','FEMALLE'):
        sex = 'F'
    if sex in ('MALE','MMALE','MM'):
        sex = 'M'

    sex_age_list[0] = sex
    sex_age_list[1] = sex_age_list[1].strip()
    return sex_age_list
def process_sex_age(df):
    df["sex_age_list"] = df["sex and age"].str.split(",").apply(clean_sex_age)
    df[['sex','age']] = pd.DataFrame(df["sex_age_list"].values.tolist(), index= df.index)
    df["age"] = pd.to_numeric(df["age"],errors='coerce')
    df.drop(columns=["sex_age_list","sex and age"],inplace=True)
    return df

In [ ]:
train = process_sex_age(train)

In [ ]:
train.head()

,id,high_BP,heart_condition_detected_2017,married,job_status and living_area,average_blood_sugar,BMI,smoker_status,TreatmentA,TreatmentB,TreatmentC,TreatmentD,stroke_in_2018,job_status,living_area,sex,age
0,16053.0,1,0,1,government?Remote,71.67,36.6,non-smoker,NaN,NaN,NaN,NaN,0,government,remote,F,61.0
1,1459.0,0,0,0,Remote?private_sector,107.95,30.4,quit,NaN,NaN,NaN,NaN,0,private_sector,remote,F,30.0
2,7678.0,1,0,1,government?Remote,76.49,42.1,active_smoker,NaN,NaN,NaN,NaN,0,government,remote,F,51.0
3,34943.0,0,0,1,government?City,113.98,57.3,quit,NaN,NaN,NaN,NaN,0,government,city,F,54.0
4,17741.0,0,0,0,private_sector?City,70.60,26.7,active_smoker,NaN,NaN,NaN,NaN,0,private_sector,city,M,27.0


In [ ]:
test.head()

,id,sex and age,high_BP,heart_condition_detected_2017,married,job_status and living_area,average_blood_sugar,BMI,smoker_status,TreatmentA,TreatmentB,TreatmentC,TreatmentD,job_status,living_area
0,33327,"F, 36",0.0,0.0,1.0,private_sector?Remote,76.05,33.4,active_smoker,NaN,NaN,NaN,NaN,private_sector,remote
1,839,"F, 40",0.0,0.0,1.0,City?government,73.77,30.1,non-smoker,NaN,NaN,NaN,NaN,government,city
2,11127,"M, 59",0.0,0.0,1.0,business_owner?Remote,62.95,30.8,None,NaN,NaN,NaN,NaN,business_owner,remote
3,20768,"33, F",0.0,0.0,1.0,private_sector?City,68.81,36.5,quit,NaN,NaN,NaN,NaN,private_sector,city
4,37774,"F, 22",0.0,0.0,0.0,private_sector?City,122.89,30.8,active_smoker,NaN,NaN,NaN,NaN,private_sector,city


In [ ]:
def merge(grp):
    df = pd.DataFrame()
    if(grp.shape[0] > 1):
        for c in grp.columns:
            value_counts = grp[c].value_counts().index.astype(grp[c].dtypes)
            if value_counts.size > 1:
                print(grp, value_counts) #Error
            elif value_counts.size == 1:
                df[c] = value_counts[0]
            else:
                df[c] = None
    else:
        df = grp.head(1)
    return df

train = train.groupby("id").apply(merge).set_index("id")

In [ ]:
profile = train.profile_report(title='Medical Record Profiling Report')
profile.to_file(output_file="train_data_summary.html")

In [ ]:
train.profile_report(style={'full_width':True})

In [ ]:
test.profile_report(style={'full_width':True})